In [1]:
# !pip install transformers --user
# !pip install pandas --user
import gc
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
def empty_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [5]:
# !pip install wandb
import wandb

In [6]:
project_name = 'bart-sum400-16epochs-2batch'

In [7]:
device = 'cuda'

In [8]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.ementa = self.data.ementa
        self.inteiro_teor = self.data.inteiro_teor

    def __len__(self):
        return len(self.ementa)

    def __getitem__(self, index):
        inteiro_teor = str(self.inteiro_teor[index])
        inteiro_teor = ' '.join(inteiro_teor.split())

        ementa = str(self.ementa[index])
        ementa = ' '.join(ementa.split())

        source = self.tokenizer.batch_encode_plus([inteiro_teor], max_length=self.source_len, pad_to_max_length=True,return_tensors='pt', truncation=True)
        target = self.tokenizer.batch_encode_plus([ementa], max_length=self.summ_len, pad_to_max_length=True,return_tensors='pt', truncation=True)

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [9]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        empty_cache()
        
        if _ % 10 == 0:
            wandb.log({"Loss do treinamento": loss.item()})

        if _ % 500 == 0:
            print(f'Época: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        empty_cache()

In [10]:
import os
os.environ['WANDB_API_KEY'] = '99e2345c917d402940bb92ff1bee8818219b8241'

In [11]:
def init_wandb(project_name):
    wandb.init(project=project_name) 
    config = wandb.config
    config.TRAIN_BATCH_SIZE = 2
    config.VALID_BATCH_SIZE = 1
    config.TRAIN_EPOCHS = 16
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 1e-4
    config.MAX_LEN = 500
    config.SUMMARY_LEN = 100

    return config

In [12]:
def init_training_pipeline(config):
    print('Iniciando pipeline...\n\n')

    model_name = 'facebook/bart-base'
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

    empty_cache()
    
    train_dataset = pd.read_csv('../../data/train/translated-train-400-2500.csv', encoding='utf-8', error_bad_lines=False, engine="python")
    train_dataset = train_dataset[['ementa','inteiro_teor']]
    print('Exemplo de textos:')
    print(train_dataset.head(), '\n\n')
    
    val_dataset = pd.read_csv('../../data/train/translated-validate-400-2500.csv', encoding='utf-8', error_bad_lines=False, engine="python")
    val_dataset = val_dataset[['ementa','inteiro_teor']]

    print(f'Dataset de treino: {train_dataset.shape}')
    print(f'Dataset de teste: {val_dataset.shape}')

    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    train_params = {'batch_size': config.TRAIN_BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
    val_params = {'batch_size': config.VALID_BATCH_SIZE, 'shuffle': False, 'num_workers': 0}

    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    
    print('Instanciando modelo...')
    model = BartForConditionalGeneration.from_pretrained(model_name)
    model = model.to(device)

    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    empty_cache()

    
    print('Inicializando Fine-Tuning utilizando o dataset de acórdãos...')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

        empty_cache()

    print('Treinamento concluído!\n\n')

    return tokenizer, model, val_loader

In [13]:
config = init_wandb(project_name)

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: thiagocmoreira (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.1
wandb: Run data is saved locally in wandb/run-20201130_180902-10qq0nig
wandb: Syncing run wandering-valley-2


In [14]:
%%time
tokenizer, model, val_loader = init_training_pipeline(config)

Iniciando pipeline...


Exemplo de textos:
                                              ementa  \
0  interlocutory appeal. Work accident. partial r...   
1  interlocutory appeal. innominate precautionary...   
2  declaration embargoes. addictions. inexistence...   
3  interlocutory appeal. timing. jurisprudential ...   
4  interlocutory appeal. illicit outsourcing. Pre...   

                                        inteiro_teor  
0  judgment\n\n6th class acv / rp\n\nThis documen...  
1  the c o rd\n\n(1st class class) gmwoc / db\n\n...  
2  judgment\n\n5th class emp / igr\n\nvisa, repor...  
3  the c o rd\n\n(6th class) gmacc\n\n/ sc / jr /...  
4  judgment\n\n6th class acv / acc\n\nvisa, repor...   


Dataset de treino: (10791, 2)
Dataset de teste: (2698, 2)
Instanciando modelo...
Inicializando Fine-Tuning utilizando o dataset de acórdãos...
Época: 0, Loss:  7.233688831329346


/home/labsuser/.local/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/home/labsuser/.local/lib/python3.6/site-packages/transformers/modeling_bart.py:1044: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


Época: 0, Loss:  1.3144252300262451
Época: 0, Loss:  0.5800948739051819
Época: 0, Loss:  1.8687855005264282
Época: 0, Loss:  0.6720576882362366
Época: 0, Loss:  1.6490108966827393
Época: 0, Loss:  0.9170570969581604
Época: 0, Loss:  0.5691471099853516
Época: 0, Loss:  1.038708209991455
Época: 0, Loss:  0.934636116027832
Época: 0, Loss:  0.6810092926025391
Época: 1, Loss:  1.1417458057403564
Época: 1, Loss:  0.1279996782541275
Época: 1, Loss:  0.35325706005096436
Época: 1, Loss:  1.0875505208969116
Época: 1, Loss:  0.37323859333992004
Época: 1, Loss:  0.558336079120636
Época: 1, Loss:  0.6432148218154907
Época: 1, Loss:  0.7721365690231323
Época: 1, Loss:  0.9349318146705627
Época: 1, Loss:  0.9958311915397644
Época: 1, Loss:  1.0020629167556763
Época: 2, Loss:  0.7845589518547058
Época: 2, Loss:  0.6361064910888672
Época: 2, Loss:  0.3804984986782074
Época: 2, Loss:  0.8081725239753723
Época: 2, Loss:  0.20903447270393372
Época: 2, Loss:  0.46028250455856323
Época: 2, Loss:  1.03375327

In [15]:
open("predictions.csv","w+")

<_io.TextIOWrapper name='predictions.csv' mode='w+' encoding='UTF-8'>

In [16]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    source_texts = []
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['source_ids'].to(device, dtype=torch.long)
            y = data['target_ids'].to(device, dtype=torch.long)
            mask = data['source_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                max_length=250,
                num_beams=2,
                repetition_penalty=1.5,
                length_penalty=1,
                early_stopping=True
            )
            
            src_texts = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in ids]
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]

            if _ % 100 == 0 and _ > 0:
                print(f'Resumos: {_} gerados')
            
#             if _ == 50:
#                 break
            
            source_texts.extend(src_texts)
            predictions.extend(preds)
            actuals.extend(target)

    return source_texts, predictions, actuals

In [17]:
def validade_and_save_predictions(val_epochs, tokenizer, model, val_loader):
    print('Gerando sumários utilizando o modelo no dataset de validação...')
    for epoch in range(val_epochs):
        source_texts, predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({ 'inteiro_teor': source_texts, 'ementa_original': actuals, 'resumo_gerado': predictions })
        final_df.to_csv('predictions.csv')
        print('CSV para análise gerado!')

In [18]:
%%time
validade_and_save_predictions(config.VAL_EPOCHS, tokenizer, model, val_loader)

Gerando sumários utilizando o modelo no dataset de validação...
Resumos: 100 gerados
Resumos: 200 gerados
Resumos: 300 gerados
Resumos: 400 gerados
Resumos: 500 gerados
Resumos: 600 gerados
Resumos: 700 gerados
Resumos: 800 gerados
Resumos: 900 gerados
Resumos: 1000 gerados
Resumos: 1100 gerados
Resumos: 1200 gerados
Resumos: 1300 gerados
Resumos: 1400 gerados
Resumos: 1500 gerados
Resumos: 1600 gerados
Resumos: 1700 gerados
Resumos: 1800 gerados
Resumos: 1900 gerados
Resumos: 2000 gerados
Resumos: 2100 gerados
Resumos: 2200 gerados
Resumos: 2300 gerados
Resumos: 2400 gerados
Resumos: 2500 gerados
Resumos: 2600 gerados
CSV para análise gerado!
CPU times: user 1h 3min 16s, sys: 1min 39s, total: 1h 4min 56s
Wall time: 42min 28s


In [19]:
generated_summaries = pd.read_csv('predictions.csv', encoding='utf-8', error_bad_lines=False, engine="python")
for index, row in generated_summaries[10:30].iterrows():
    print(f'Exemplo {index}', '\n')
    print('Inteiro teor:', row['inteiro_teor'], '\n\n')
    print('Ementa original:', row['ementa_original'], '\n\n')
    print('Sumário gerado:', row['resumo_gerado'], '\n\n')

Exemplo 10 

Inteiro teor: the c o rd (ac. (8th class) gmmea / hagb / acnv This document of interlocutory appeal was reviewed, reported and discussed in resource of magazine n ° tst-airr-127740-66.2008.5.02.000, in which joyce de lime is aggravating and silva and silva associate lawyers and professional cooperative of the professionals are aggravated credit and collection - cccoop. the claimant lodges an interlocutory appeal (pages 2/7) against the dispatch of pages. 95, which denied following up on its review appeal. there was no counter-submission. remittance of the records to the public labor ministry is waived, according to art. 83 of the ritst. is the report. vote 1 - I am aware of the interlocutory appeal because the legal requirements for admissibility are met. 2 - merit of the union's framework, the regional denied following up the appeal of a magazine with fulcrum in art. 896, § 6, of clt. the aggravating factor, reiterating the reasons for its appeal to the magazine, claims t

In [20]:
predictions = pd.read_csv('predictions.csv', encoding='utf-8')

In [21]:
# !pip install rouge-score tqdm --user
from tqdm import tqdm_notebook
# import time
from rouge_score import rouge_scorer, scoring
from typing import List, Dict

In [22]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [23]:
ROUGE_KEYS = ["rouge1", "rouge2", "rougeL"]
def calculate_rouge(output_lns: List[str], reference_lns: List[str]) -> Dict:
    scorer = rouge_scorer.RougeScorer(ROUGE_KEYS, use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()

    for reference_ln, output_ln in tqdm_notebook(zip(reference_lns, output_lns)):
        scores = scorer.score(reference_ln, output_ln)
        aggregator.add_scores(scores)

    result = aggregator.aggregate()
    return {k: v.mid.fmeasure for k, v in result.items()}

In [24]:
metrics = calculate_rouge(predictions['ementa_original'], predictions['resumo_gerado'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [25]:
metrics

{'rouge1': 0.7100650830985114,
 'rouge2': 0.5609695523810816,
 'rougeL': 0.6582767049691132}

In [26]:
# model_archive_name = 'bart-sum400-model-8'
# print('Salvando modelo treinado...')
# model.save_pretrained(model_archive_name)
# print(f'Modelo salvo na pasta {model_archive_name}!\n')